# RNN NUMPY

**YBIGTA 10기 손진원**

In [1]:
import numpy as np

In [2]:
def softmax(x):
    if x.ndim==1:
        const = np.max(x)
        expo = np.exp(x - const)
        res = expo / np.sum(expo)
        return(res)
    else:
        const = np.apply_along_axis(arr=x,axis=x.ndim-1,func1d=np.max)
        empty = np.ndarray(shape=x.shape)
        for i in range(x.ndim):
            empty[i]=const[i]
        
        expo = np.exp(x - empty)
        res = np.apply_along_axis(arr=expo, axis=x.ndim-1, func1d=lambda x : x/np.sum(x))
        return(res)

In [3]:
sentence = (" Then you will know the truth, and the truth will set you free ")
idx2char = list(set(sentence))  # 단어장
char2idx = {c:i for i,c in enumerate(idx2char)}
char2idx_to_string = {i:c for i,c in enumerate(idx2char)}
sentence_idx = [char2idx[c] for c in sentence]

In [4]:
sqt_length=10    # time_step 단위

dataX=[]
# sentence_idx=sentence
for i,c in enumerate(sentence_idx):
    x = sentence_idx[i:i+sqt_length]
    if len(x) == 10:
        dataX.append(x)
    else:
        pass
    
X_data = np.array(dataX[:-1])
Y_data = np.array(dataX[1:])
print(X_data.shape)
print(Y_data.shape)   # 53개의 데이터

(53, 10)
(53, 10)


In [5]:
def one_hot(x, num_chars):
    Iden=np.eye(num_chars,dtype=np.int32)
    return(Iden[x])

In [6]:
X = one_hot(X_data, 19)
Y = Y_data

![rnn_link](http://www.wildml.com/wp-content/uploads/2015/09/rnn.jpg)

In [7]:
# no_batch
class RNN_numpy:
    def __init__(self, word_dim, hidden_dim=5, bptt_truncate=4):  # bptt_truncate : 특정 스텝에서 계산할 이전 스텝의 수
        # 인스턴스 변수 설정
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        
        # 초깃값 임의로 설정
        self.U = np.random.uniform(-np.sqrt(1./word_dim),np.sqrt(1./word_dim),(hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim),np.sqrt(1./hidden_dim),(word_dim,hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim),np.sqrt(1./hidden_dim),(hidden_dim, hidden_dim))
        
    def forward(self,x):
        # The total number of time steps
        T = len(x)
        # hidden_states 정의, 더해지는 s_1 을 계산하기 위하여 s_0의 공간을 만드는 것이다.
        s = np.zeros((T+1,self.hidden_dim))
        s[-1] = np.zeros(self.hidden_dim)      # S_0를 0으로 담아놓는 공간.
        # 아웃풋을 담아놓는 공간.
        o = np.zeros((T,self.word_dim))  
        
        # 매번 스텝(t) 마다
        for t in np.arange(T):
            # self.U[:,x[t]] == np.dot(self.U, x[t]) 이다. x[t]가 one_hot_vector이기 때문이다.
            s[t] = np.tanh(self.U[:,np.argmax(x[t])] + np.dot(self.W,s[t-1]))
            o[t] = softmax(np.dot(self.V,s[t]))   # 확률벡터로 나온다.
            
        return o,s
    
    def predict(self,x):
        o, s = self.forward(x)
        return np.argmax(o,axis=1)
    
    def total_loss(self,x,y):  # y는 실제 값. y.shape = batch * time_steps
        L = 0
        o =self.forward(x)[0]
        for i in range(len(y)):
           L+= -np.sum(np.log(o[i][y[i]]))
        return L
    
    def loss(self, x, y):
        N = y.shape[0]     # batch_size로 나누어준다.
        return self.total_loss(x,y)/N

### 값이 어떻게 나오나 확인해보자

In [8]:
rnn = RNN_numpy(word_dim=19,hidden_dim=5)

In [9]:
o,s = rnn.forward(X[0])

In [10]:
o[0]  # X[0]에 대한 첫 번째 아웃풋이다. 첫 번째 글자 다음으로 나올 수 있는 19개의 character가 확률로 나타난다.

array([ 0.0527257 ,  0.05334002,  0.04904424,  0.0530377 ,  0.05270872,
        0.05222811,  0.0480116 ,  0.05388399,  0.04897119,  0.05616717,
        0.05820783,  0.05202417,  0.05385329,  0.05654336,  0.05510058,
        0.04738768,  0.05152483,  0.05638448,  0.04885534])

## SGD & BPTT(Backpropagation Throught Time)
-  RNN에서는 Backpropagation Through Time (BPTT) 을 사용한다. 그 이유는 각 파라미터들이 네트워크의 매 시간 스텝마다 공유되기 때문이다. 즉, 각 시간 스텝의 출력단에서의 gradient는 현재 시간 스텝에서의 계산에만 의존하는 것이 아니라 이전 시간 스텝에도 의존한다.
- 각 파라미터를 공유하기 때문에 각 타임스텝에서 발생하는 미분값들을 모두 더해준다.
- 여기서 공유된다 함은 U, V, W가 모든 히든레이어에 동일한 모두로 연결되어있음을 의미한다.
- bptt_truncate => 입력 시퀀스가 긴 경우에 깊은 layer들에 모두 backpropagate해 주어야 한다.(dL/dW를 구하는 경우에) 이럴 경우 계산이 너무 복잡해지므로 실제 상황에서는 보통 backpropagation through time을 할 시간 스텝을 적당한 숫자로 정해준다.

![link1](http://www.wildml.com/wp-content/uploads/2015/10/rnn-bptt-with-gradients.png)

## 아래는 bptt_truncate = 4로 제한 했을 때 $dL \over dW$를 구하는 방법이다.

## 학습이 진행되는 순서는 아래와 같다.

In [11]:
for t in np.arange(len(Y[0]))[::-1]:
    for bptt_step in np.arange(max(0, t-4), t+1)[::-1]:
        print('time_step:',t, 'and bptt_step',bptt_step)

time_step: 9 and bptt_step 9
time_step: 9 and bptt_step 8
time_step: 9 and bptt_step 7
time_step: 9 and bptt_step 6
time_step: 9 and bptt_step 5
time_step: 8 and bptt_step 8
time_step: 8 and bptt_step 7
time_step: 8 and bptt_step 6
time_step: 8 and bptt_step 5
time_step: 8 and bptt_step 4
time_step: 7 and bptt_step 7
time_step: 7 and bptt_step 6
time_step: 7 and bptt_step 5
time_step: 7 and bptt_step 4
time_step: 7 and bptt_step 3
time_step: 6 and bptt_step 6
time_step: 6 and bptt_step 5
time_step: 6 and bptt_step 4
time_step: 6 and bptt_step 3
time_step: 6 and bptt_step 2
time_step: 5 and bptt_step 5
time_step: 5 and bptt_step 4
time_step: 5 and bptt_step 3
time_step: 5 and bptt_step 2
time_step: 5 and bptt_step 1
time_step: 4 and bptt_step 4
time_step: 4 and bptt_step 3
time_step: 4 and bptt_step 2
time_step: 4 and bptt_step 1
time_step: 4 and bptt_step 0
time_step: 3 and bptt_step 3
time_step: 3 and bptt_step 2
time_step: 3 and bptt_step 1
time_step: 3 and bptt_step 0
time_step: 2 a

In [12]:
# no_batch
class RNN_numpy:
    def __init__(self, word_dim, hidden_dim=5, bptt_truncate=4):
        # 인스턴스 변수 설정
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        
        # 초깃값 임의로 설정
        self.U = np.random.uniform(-np.sqrt(1./word_dim),np.sqrt(1./word_dim),(hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim),np.sqrt(1./hidden_dim),(word_dim,hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim),np.sqrt(1./hidden_dim),(hidden_dim, hidden_dim))
        
    def forward(self,x):
        # The total number of time steps
        T = len(x)
        # hidden_states 정의, 더해지는 s_1 을 계산하기 위하여 s_0의 공간을 만드는 것이다.
        s = np.zeros((T+1,self.hidden_dim))
        s[-1] = np.zeros(self.hidden_dim)      # s[0] 아닌가?
        # 아웃풋을 담아놓는 공간.
        o = np.zeros((T,self.word_dim))  
        
        # 매번 스텝(t) 마다
        for t in np.arange(T):
            # self.U[:,x[t]] == np.dot(self.U, x[t]) 이다. x[t]가 one_hot_vector이기 때문이다.
            s[t] = np.tanh(self.U[:,np.argmax(x[t])] + np.dot(self.W,s[t-1]))
            o[t] = softmax(np.dot(self.V,s[t]))   # 확률벡터로 나온다.
            
        return o,s
    
    def predict(self,x):
        o, s = self.forward(x)
        return np.argmax(o,axis=1)
    
    def total_loss(self,x,y):  # y는 실제 값. y.shape = batch * time_steps
        L = 0
        o =self.forward(x)[0]
        for i in range(len(y)):
           L+= -np.sum(np.log(o[i][y[i]]))
        return L
    
    def loss(self, x, y):
        N = y.shape[0]     # batch_size로 나누어준다.
        return self.total_loss(x,y)/N
    
    def bptt(self, x, y):
        T = len(x)   # y는 일단 no_batch mode
        o, s = self.forward(x)
        dU = np.zeros(self.U.shape)
        dV = np.zeros(self.V.shape)
        dW = np.zeros(self.W.shape)
        # one_hot에 해당되는 애들만 가져오기
        delta_o = o
        delta_o[np.arange(len(y)),y] -= 1
        
        # 마지막 시간스텝부터 dV를 계산한다.
        for t in np.arange(T)[::-1]:
            dV += np.outer(delta_o[t],s[t].T)   # outer 계산..  행렬 미분 값이 outer로 계산이 되나보다..
            # delta_t 는 전달되는 미분계수들.
            delta_t = np.dot(self.V.T,delta_o[t]) * (1-(s[t]**2))   
            
            # 먼저 dV가 계산 되고, 이후에 dW가 계산된다. 
            # dW는 해당 타임스텝에서 bptt_truncate로 제한된 이전 타임스텝까지만 계산한다.(너무 많은 계산을 피하기 위해)
            # dW가 계산되면서 이어 dU도 계산이 된다.
            for bptt_step in np.arange(max(0,t-self.bptt_truncate),t+1)[::-1]:
            
                dW += np.outer(delta_t, s[bptt_step-1])
                dU[:,np.argmax(x[bptt_step])] += delta_t 
                # update delta for next step
                delta_t = np.dot(self.W.T,delta_t) * (1-s[bptt_step-1]**2)
                
        return [dU,dV,dW]

## 학습해보기! 구현되었다! ㅎㅎ

In [37]:
del rnn # 앞서 정의되었던 클래스를 지우고

In [38]:
rnn = RNN_numpy(word_dim=19,hidden_dim=5)

In [39]:
iters_num = 100000
learning_rate = 0.0001

for j in range(iters_num):
    # 손실을 구하고
    loss = rnn.loss(x=X[0],y=Y[0])
    # 기울기를 구해서
    grad = rnn.bptt(x=X[0],y=Y[0])
    # 업데이트 한다 ㅎㅎ
    rnn.U -= learning_rate*grad[0]
    rnn.V -= learning_rate*grad[1]
    rnn.W -= learning_rate*grad[2]

    accuracy = np.mean(rnn.predict(X[0])==Y[0])
    
    if j%1000==0:
        print('loss : ',loss,'train_accuracy : ', accuracy)
    
    if accuracy >= 0.90: break
    
        

loss :  2.9473000052 train_accuracy :  0.1
loss :  2.88077854472 train_accuracy :  0.3
loss :  2.80259030972 train_accuracy :  0.3
loss :  2.69969929126 train_accuracy :  0.3
loss :  2.56408998391 train_accuracy :  0.4
loss :  2.39088117389 train_accuracy :  0.5
loss :  2.17755311242 train_accuracy :  0.5
loss :  1.95059477809 train_accuracy :  0.5
loss :  1.74827718879 train_accuracy :  0.5
loss :  1.58244566955 train_accuracy :  0.5
loss :  1.44825072098 train_accuracy :  0.5
loss :  1.33782637187 train_accuracy :  0.5
loss :  1.24467913357 train_accuracy :  0.6
loss :  1.16411904876 train_accuracy :  0.7
loss :  1.09286273704 train_accuracy :  0.8
loss :  1.02865744109 train_accuracy :  0.8
loss :  0.969942435585 train_accuracy :  0.8


## Vanishing Gradient 문제
- $\frac{\partial E_3}{\partial W} = \sum_{k=0}^{3}{\frac{\partial E_3}{\partial \hat{y_3}} \frac{\partial \hat{y_3}}{\partial s_3} \left( \prod_{j=k+1}^{3}{\frac{\partial s_j}{\partial s_{j-1}}} \right) \frac{\partial s_k}{\partial W}}$
- $d{S}_{n} \over d{S}_{1}$ = $d{S}_{n} \over d{S}_{n-1}$ $d{S}_{n-1} \over d{S}_{n-2}$ $d{S}_{n-3} \over d{S}_{n-4}$ $...$ $d{S}_{2} \over d{S}_{1}$  와 같이 연쇄법칙으로 표현된다.
- 이것은 백터의 백터에 대한 미분이기 때문에 결과적으로 행렬로 표현이된다. 이 행렬은 Jacobian matrix라 불리고, 그 크기는 1을 넘지 않는다.
- 결과적으로 위의 연쇄법칙에서 구성되는 야코비안 행렬의 원소들은 $(-1,1)$사이에 있는 값일 것이고, 이것들이 몇 번 곱해지면 그 크기는 지수적으로(매우 빠르게) 0을 향해 수렴하게 된다.
- 이때 Gradient가 사라지는(Vanish) 문제가 발생하게 된다.
- 만약 자코비안 행렬 안의 값들이 크다면 activation 함수와 네트워크 파라미터 값에 따라 gradient가 사라지는게 아니라 오히려 지수 함수로 증가하는 경우도 가능할 수 있다. 이런 문제를 exploding gradient라 한다.
- **해결책** : 초깃값을 **잘** 지정, 활성화함수로 tanh 나 sigmoid 대신에 Relu를 이용. 혹은 LSTM이나 GRU **구조**를 이용

## LSTM과 GRU
- LSTM: 1997년 스위스의 Sepp Hochreiter와 ürgen Schmidhube에 의해 제안
- LSTM: 자연어처리에서 넓게 활용 중
- GRU: 2014년 뉴욕대 조경현 교수에 의해 제안
- GRU: LSTM보다 간단한 구조

## LSTM의 원리
- 몇 가지 게이트를 추가함으로써 hidden state를 다르게 계산한다. 이를 LSTM 모듈이라 한다.
- 초기 버전 $s_t = \tanh(U_tx_t+W_t{s}_{t-1})$ 


### LSTM 모듈이 동작하는 방식

![link2](https://deeplearning4j.org/img/greff_lstm_diagram.png)


$\begin{align} 
i & = \sigma(x_tU^i + s_{t-1} W^i) \\ 
f & = \sigma(x_t U^f +s_{t-1} W^f) \\ 
o & = \sigma(x_t U^o + s_{t-1} W^o) \\ 
g & = \tanh(x_t U^g + s_{t-1}W^g) \\ 
c_t & = c_{t-1} \circ f + g \circ i \\ 
s_t & = \tanh(c_t) \circ o 
\end{align}$


${i}$ : 입력 게이트, ${f}$ : 까먹음 게이트, ${o}$ : 출력 게이트이며, 각 게이트는 모두 서로 다른 행렬 ${U}$와 ${W}$를 갖는다.

${g}$는 현재 입력과 이전 hidden state 값을 기반으로 계산된 현재 hidden state의 후보값

${c}_{t}$는 메모리로 이전 메모리에서 쓰지 않을 정보를 ${f}$ 게이트를 통해 걸러내고, ${i}$ 게이트로 $g$로부터 받아들일 정보만 통과시킨다. 그리고 $tanh$ 를 거친 ${c}_{t}$는 게이트 ${o}$를 통해 또 한 번 걸러져서 출력되게 된다.

- 위 과정은 **black box** 형태로 진행이 된다.

- 입력 게이트를 모두 1로, 까먹음 게이트를 모두 0(이전 값은 기억하지 않음)으로, 출력 게이트를 모두 1로 둔다면 기본적인 RNN과 거의 같게 된다.

- ${c}_{t}$에서 연쇄미분이 작동하게 되는데, 이때 f가 그것의 비가 된다. ex) ${dc}_{t} \over {dc}_{t-1}$=$f$,$\quad$ **${f}$는 게이트이므로 1과 0으로 구성된다.** 따라서 기울기가 1 또는 0이 되면서 필요한 정보는 유지되고, 불필요한 정보는 버려진다. 또한 이때 vanishing gradient가 문제가 해결된다.

### GRU 모듈이 동작하는 방식

$\begin{aligned} 
z &= \sigma(x_tU^z + s_{t-1} W^z) \\ 
r &= \sigma(x_t U^r +s_{t-1} W^r) \\ 
h &= tanh(x_t U^h + (s_{t-1} \circ r) W^h) \\ 
s_t &= (1 - z) \circ h + z \circ s_{t-1} 
\end{aligned}$

${r}$은 리셋 게이트로 새로운 입력을 이전 메모리와 어떻게 합칠지를 정해주고
, ${z}$는 업데이트 게이트로 이전 메모리를 얼마만큼 기억할지를 정해준다.
리셋 게이트를 전부 1로, 업데이트 게이트를 전부 0으로 설정하면 기본적인 RNN과 거의 같을 것이다.

${h}$는 새로운 hidden state의 후보이다.
${s}_{t}$는 후부 hidden state에서 (1-z)만큼, 이전 state에서 z만큼 반영된다.

### LSTM과 GRU의 차이점
- GRU의 게이트가 하나 더 적다.
- 출력값을 계산할 때 추가적인 비선형 함수를 적용하지 않는다.
- 게이트가 더 적은 GRU가 LSTM에 비해 더 적은 모수를 가진다.(상대적으로 더 빠르게 학습)
- 성능면에서 어떤 것이 더 좋은지는 케.바.케

## LSTM과 GRU numpy 구현은. ㅎㅎ
## 그냥 모듈 쓰세요.

**Reason for vanishing gradient problem** : In a multi-layer network, gradients for deeper layers are calculated as products of many gradients (of activation functions). When those gradients are small or zero, it will easily vanish. (On the other hand, when they’re bigger than 1, it will possibly explode.) So it becomes very hard to calculate and update.

**Solution 1)** Use activation functions which have ‘good’ gradient values. Not ZERO over a reasonable amount of range, Not that small, Not that big..

**Solution 2)** Use gating(pass or block, or in other words, 1 or 0) function, not activation function. And train the ‘combination’ of all those gates. Doing this, no matter how ‘deep’ your network is, or how ‘long’ the input sequence is, the network can remember those values, as long as those gates are all 1 along the path. <- This is how LSTM/GRU did the job.